# Chapter 15: OpenCV

**OpenCV** provides many image processing and computer vision tools.  While written in C++, it provides a fairly extensive Python interface, enabling it to be used as a Python toolbox, see: [https://docs.opencv.org/4.5.3/](https://docs.opencv.org/4.5.3/).  Use OpenCV for the many powerful image operations such as segmentation, calibration, warping, and even applying deep neural networks to images.  Plotting is fairly limited, and more powerful plotting is available in Matplotlib, see  [Chapter 16: Matplotlib](Chapter_16_Matplotlib.ipynb).

For installation, see [Chapter 13](Chapter_13_Numpy.ipynb).

## Image Operations with OpenCV

Images are another data type for which Numpy arrays are useful.  We will use OpenCV to read and process an image.  

In [1]:
import cv2 as cv
import numpy as np
img = cv.imread('.Images/book.png', -1)

Here I have provided the full path from where Python is running to the `book.png` image in this repo.  The `-1` flag reads in raw data without changing the format or number of channels.  You'll need to adjust your path to point to this image from where you are running Python.  Now let's examine the image we just read in:

In [2]:
type(img)

numpy.ndarray

If you see an output `NoneType`, then `img` was not successfully loaded.  You'll need to adjust the path to the image.  If you want to know the path of where Python is currently running use this:

In [3]:
import os
os.getcwd() 

'd:\\DMorris\\Repos\\teaching\\python_intro'

I assume you have adjusted the path and successfully read in the image, and it is a `numpy.ndarray`.  Let's examine its size and the pixel type:

In [4]:
img.shape

(247, 160, 3)

In [5]:
img.dtype 

dtype('uint8')

We see that it is a 3D array; the `.shape` is a tuple with array dimension sizes, in this case: (height, width, nchannels).  The three channels are (blue, green, red), as this is the default format for OpenCV, which differs from the more usual (red, green, blue) format.  The data type is an unsigned 8-bit type (`uint8`) which stores numbers from 0 to 255 inclusive.  We can extract the 3 components of a sample single pixel as follows:

In [6]:
img[22,28,:]

array([212, 187, 143], dtype=uint8)

Or we can drop the trailing colon to get the same output:

In [7]:
img[22,28]  

array([212, 187, 143], dtype=uint8)

These three values are the (blue, green, red) values for that pixel.  Review Numpy [indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html) if this isn't clear.

Let's say our goal is to find the horizontal image gradient of the grayscale version of this image.  Here are the steps we would take.

In [8]:
imgf = img / 255 

This converts a floating point array and scales to a range of 0 to 1, which is the usual range for floating point images.  It is best to use floating point rather than unsigned integers because the gradient can be negative or could be larger than the maximum `uint8` value of 255.  Operating in `uint8` will result in gradients being clipped.  Confirming our new data type:

In [9]:
imgf.dtype

dtype('float64')

Next, convert the image to grayscale with `cvtColor`, which is a general purpose colorspace transformation utility:

In [10]:
imgf_gray = cv.cvtColor(imgf.astype(np.float32), cv.COLOR_BGR2GRAY)  # Convert 3-channel color to single-channel grayscale

If you do `help(cv.cvtColor)` you will see that `cvtColor` requires floating-bit arrays be 32-bit, while `img` is 64-bit, so we passed in a 32-bit version of `img`.  This grayscale image is the same size, except with a single channel:

In [11]:
imgf_gray.shape

(247, 160)

Finally, let's apply a Sobel operator to obtain the image gradient in x:

In [12]:
imgf_grad_x = cv.Sobel(imgf_gray, cv.CV_32F, 1, 0, ksize=5)

To see the meaning of the parameters, use: `help(cv.Sobel)`.  Next, let's see how to plot our result.

## Plotting with OpenCV
OpenCV has some graphical display functions including showing images.  For instance:

In [13]:
cv.imshow('Image',img)
cv.waitKey(1)           # This gives OpenCV 1 millisecond to render the image

-1

This will show an image in a **separate window** that should look like this:

![Image](.Images/show_book.png)

Find the window (which might be underneath this document window) and when you are done, delete it with:

In [14]:
cv.destroyAllWindows()

Let's say we want to draw a circle at the image center.  Using the OpenCV `circle` function, this will actually change the pixel values, so if we want to avoid changing our `img` variable we could make a copy of it just for plotting as follows:

In [15]:
img_cp = img.copy()
img_cen_xy = (img_cp.shape[1]//2, img_cp.shape[0]//2)  # Integer division and order (x,y) 
cv.circle( img_cp, img_cen_xy, radius=15, color=(0,0,255), thickness=2 )
cv.imwrite('.Images/book_circle.png',img_cp)
cv.imshow('Image with Circle', img_cp)
cv.waitKey(1)

-1

Again, find this window to see the output which should look like this:

![Image](.Images/book_circle.png)

Notice the pixelation as the circle is part of the image.  When done, delete the window with:

In [16]:
cv.destroyAllWindows()

## Connected Components
The OpenCV function: `connectedComponentsWithStats` is quite useful.  You can find detailed information on its arguments using `help(cv.connectedComponentsWithStats)`.  Also below is an example of its use.  First define an 8-bit image with binary components:

In [43]:
nimg = ( np.array([1,1,0,1,1,1])[:,None] * np.array([1,1,0,1,1,1,1,1]) ).astype(np.uint8)
nimg

array([[1, 1, 0, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1, 1]], dtype=uint8)

Now find connected components and details for each component:

In [44]:
cc = cv.connectedComponentsWithStats( nimg )
print('Image with each pixel assigned a label value:')
print(cc[1])
print('Label, Bounding Box,  Npix,  Centroid')
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
for i in range(cc[0]): # cc[0] is the number of connected components
    print(f'{i:3}      {cc[2][i,:4]}   {cc[2][i,4]:4}    {cc[3][i,:]}')
print('Bounding box format: [left,top,width,height]')

Image with each pixel assigned a label value:
[[1 1 0 2 2 2 2 2]
 [1 1 0 2 2 2 2 2]
 [0 0 0 0 0 0 0 0]
 [3 3 0 4 4 4 4 4]
 [3 3 0 4 4 4 4 4]
 [3 3 0 4 4 4 4 4]]
Label, Bounding Box,  Npix,  Centroid
  0      [0 0 8 6]     13    [ 2.923  2.231]
  1      [0 0 2 2]      4    [ 0.500  0.500]
  2      [3 0 5 2]     10    [ 5.000  0.500]
  3      [0 3 2 3]      6    [ 0.500  4.000]
  4      [3 3 5 3]     15    [ 5.000  4.000]
Bounding box format: [left,top,width,height]


We can see that the components of the output `cc` are:
```
cc[0]:  1x1: Number of connected components: N
cc[1]:  Input image size with each pixel given its connected component label
cc[2]:  Nx5: Bounding box and number of pixels in each component
cc[3]:  Nx2: Centroid for each component
```

___
### [Outline](README.md), Next: [Chapter 16: Matplotlib](Chapter_16_Matplotlib.ipynb)